In [1]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle
import random
import pdb
from torch.utils.data import DataLoader
import logging
import itertools
import unicodedata
import string
import re
import random
import argparse
from torch import optim
import time
import os
from bleu_score import BLEU_SCORE
# from models_viet import EncoderRNN, AttentionDecoderRNN, DecoderRNN
from load_dataset_viet import *
# from define_training_viet import *
import torchtext
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = torch.device('cuda')

In [3]:
class Vietnamese(Dataset):
    def __init__(self, df, val = False):
        self.df = df
        self.val = val
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx,:]['en_idized']
        viet = self.df.iloc[idx,:]['vi_idized']
        en_len = self.df.iloc[idx,:]['en_len']
        vi_len = self.df.iloc[idx,:]['vi_len']
        if self.val:
            en_data = self.df.iloc[idx,:]['en_data'].lower()
            return [viet,english,vi_len,en_len,en_data]
        else:
            return [viet,english,vi_len,en_len]
    
    
def vocab_collate_func(batch):
    MAX_LEN_EN = 48
    MAX_LEN_VI = 48
    en_data = []
    vi_data = []
    en_len = []
    vi_len = []
    for datum in batch:
        en_len.append(datum[3])
        vi_len.append(datum[2])
    max_batch_length_en = max(en_len)
    max_batch_length_vi = max(vi_len)
    if max_batch_length_en < MAX_LEN_EN:
        MAX_LEN_EN = max_batch_length_en
    if max_batch_length_vi < MAX_LEN_VI:
        MAX_LEN_VI = max_batch_length_vi
    # padding
    for datum in batch:
        if datum[2]>MAX_LEN_VI:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN_VI]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN_VI - datum[2])),
                                mode="constant", constant_values=PAD_IDX)
        if datum[3]>MAX_LEN_EN:
            padded_vec_s2 = np.array(datum[1])[:MAX_LEN_EN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_LEN_EN - datum[3])),
                                mode="constant", constant_values=PAD_IDX)
        en_data.append(padded_vec_s2)
        vi_data.append(padded_vec_s1)
    vi_data = np.array(vi_data)
    en_data = np.array(en_data)
    vi_len = np.array(vi_len)
    en_len = np.array(en_len)
#     sorted_vi_len = np.argsort(-vi_len)
#     vi_data = vi_data[sorted_vi_len]
#     en_data = en_data[sorted_vi_len]
#     vi_len = vi_len[sorted_vi_len]
#     en_len = en_len[sorted_vi_len]
#     print(en_len)
    vi_len[vi_len>MAX_LEN_VI] = MAX_LEN_VI
    en_len[en_len>MAX_LEN_EN] = MAX_LEN_EN
        
    return [torch.from_numpy(vi_data), torch.from_numpy(en_data),
            torch.from_numpy(vi_len), torch.from_numpy(en_len)]

def convert_idx_2_sent(tensor, lang_obj):
    word_list = []
    for i in tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

def convert_id_list_2_sent(list_idx, lang_obj):
    word_list = []
    if type(list_idx) == list:
        for i in list_idx:
            if i not in set([EOS_token]):
                word_list.append(lang_obj.index2word[i])
    else:
        for i in list_idx:
            if i.item() not in set([EOS_token,SOS_token,PAD_IDX]):
                word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

In [4]:
def vocab_collate_func_val(batch):
    return [torch.from_numpy(np.array(batch[0][0])).unsqueeze(0), torch.from_numpy(np.array(batch[0][1])).unsqueeze(0),
            torch.from_numpy(np.array(batch[0][2])).unsqueeze(0), torch.from_numpy(np.array(batch[0][3])).unsqueeze(0),batch[0][4]]

In [5]:
# MAX_LEN = 48
train,val,en_lang,vi_lang = train_val_load(48, "", '/scratch/ark576/machine_translation_data/')

In [6]:
def shuffle_sorted_batches(train, bs):
    batch_samp_list = list(BatchSampler(SequentialSampler(train), bs, drop_last = False))
    np.random.shuffle(batch_samp_list)
    batch_samp_list_merged = list(itertools.chain(*batch_samp_list))
    return train.iloc[batch_samp_list_merged,:]

In [7]:
train.head()

,en_data,vi_data,en_tokenized,vi_tokenized,en_idized,vi_idized,en_len,vi_len
0,Rachel Pike : The science behind a climate hea...,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[rachel, pike, :, the, science, behind, a, cli...","[khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]","[8225, 2, 137, 4, 150, 573, 9, 15, 3717, 1]","[7, 1433, 5, 3054, 4, 13, 1]",10,7
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[in, 4, minutes, ,, atmospheric, chemist, rach...","[trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q...","[12, 5080, 264, 7, 20, 4552, 8225, 2, 3006, 9,...","[23, 812, 253, 8, 1043, 1342, 20, 8510, 2, 161...",51,57
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[i, &apos;d, like, to, talk, to, you, today, a...","[tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ...","[23, 247, 16, 13, 272, 13, 33, 273, 100, 4, 14...","[26, 152, 47, 11, 38, 149, 4, 92, 787, 9, 15, ...",28,26
3,Headlines that look like this when they have t...,Có những dòng trông như thế_này khi bàn về biế...,"[headlines, that, look, like, this, when, they...","[có, những, dòng, trông, như, thế_này, khi, bà...","[6, 11, 24, 16, 21, 49, 35, 31, 13, 36, 51, 15...","[21, 15, 530, 419, 17, 193, 34, 488, 4, 1227, ...",32,25
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[they, are, both, two, branches, of, the, same...","[cả, hai, đều, là, một, nhánh, của, cùng, một,...","[35, 55, 402, 118, 3364, 5, 4, 154, 260, 5, 20...","[41, 273, 70, 16, 5, 3618, 9, 178, 5, 198, 23,...",14,16


In [8]:
bs_dict = {'train':128,'validate':1, 'train_val':1,'val_train':128}
# train_used = shuffle_sorted_batches(train_sorted, bs_dict['train'])
# train_used = train.iloc[:50]
train_used = train
collate_fn_dict = {'train':vocab_collate_func, 'validate':vocab_collate_func_val,\
                   'train_val':vocab_collate_func_val,'val_train':vocab_collate_func}
transformed_dataset = {'train': Vietnamese(train_used),
                       'validate': Vietnamese(val, val = True),
                       'train_val':Vietnamese(train.iloc[:50], val = True),
                       'val_train':Vietnamese(val)
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=collate_fn_dict[x],
                    shuffle=True, num_workers=0) for x in ['train', 'validate', 'train_val','val_train']}

In [9]:
# train.head()

In [10]:
train_data_sample = next(iter(dataloader['validate']))

In [11]:
train_data_sample[2]

tensor([16])

In [12]:
convert_id_list_2_sent(train_data_sample[1][0], en_lang)

'this is the UKN river , which serves as a part of the border between north korea and china .'

In [13]:
train_data_sample[0].shape

torch.Size([1, 16])

In [14]:
train_data_sample[1].shape

torch.Size([1, 21])

In [15]:
train_data_sample[2]

tensor([16])

In [16]:
train_data_sample[3]

tensor([21])

In [17]:
for t in zip(train_data_sample[1],train_data_sample[0]):
    en_sent = []
    vi_sent = []
    for i in t[0]:
        en_sent.append(en_lang.index2word[i.item()])
    for i in t[1]:
        vi_sent.append(vi_lang.index2word[i.item()])
    print('English:',(' ').join(en_sent))
    print('Viet:',(' ').join(vi_sent))
    print('-*'*50)

English: this is the UKN river , which serves as a part of the border between north korea and china . EOS
Viet: đây là sông áp lục nó là biên_giới tự_nhiên giữa bắc triều_tiên và trung_quốc . EOS
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*


In [18]:
vi_lang.n_words

13294

In [19]:
en_lang.n_words

16031

In [20]:
# class OldEncoderRNN(nn.Module):
#     def __init__(self, input_size, hidden_size,bi):
#         super(OldEncoderRNN, self).__init__()
#         self.bi=bi
#         if self.bi:
#             self.mul=2
#         else:
#             self.mul=1
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size,batch_first=True,bidirectional=self.bi)

#     def forward(self, input, hidden):
#         embedded = self.embedding(input)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         last_layer = output[:,-1,:self.hidden_size]
#         first_layer = output[:,0,self.hidden_size:]
#         hidden = torch.cat([first_layer,last_layer],dim=1).unsqueeze(0)
#         return output, hidden

#     def initHidden(self,bs):
#         return torch.zeros(self.mul, bs, self.hidden_size).to(device)
    
    
# class OldAttentionDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size,bi, MAX_LEN,n_layers = 1, attention_type = None):
#         super(OldAttentionDecoderRNN, self).__init__()
#         self.mul=2
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.dropout = nn.Dropout(p=0.2)
#         self.gru = nn.GRU(hidden_size*2 + hidden_size, hidden_size*2,batch_first=True, num_layers = n_layers)
        
#         self.attention_type = attention_type
#         if self.attention_type is not None:
#             self.attn = nn.Linear(self.hidden_size*2, self.hidden_size*2)
#             self.attn_drop = nn.Dropout(p = 0.5)
#         else:
#             self.attn = nn.Linear(self.hidden_size * 2, MAX_LEN)
        
# #         self.attn_combine = nn.Linear(self.hidden_size * self.mul+self.hidden_size, self.hidden_size)
        
#         self.out = nn.Linear(self.mul*hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden,encoder_outputs):
#         bss = input.size(0)
#         output = self.embedding(input)
#         output = self.dropout(output)
#         att_out = self.attn_drop(self.attn(hidden[-1])).unsqueeze(-1)
#         if self.attention_type is not None:
#             attn_wts = F.softmax(torch.bmm(encoder_outputs,att_out),dim = 1)
#             attn_applied = torch.bmm(encoder_outputs.transpose(1,2),attn_wts).transpose(1,2)
#         else:
#             att_out = F.softmax(self.attn(cat),dim=1)
#             attn_applied = torch.bmm(att_out,encoder_outputs)
#         attn_cat = torch.cat((output, attn_applied), 2)
#         output = F.relu(attn_cat)
#         output, hidden = self.gru(output, hidden)
#         output = self.out(output.squeeze(dim=1))
#         output = self.softmax(output)

#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(self.mul, bs, self.hidden_size).to(device)

In [21]:
a = torch.from_numpy(np.array([2,4,5,6,3,2,8,3,2]))

In [22]:
b = torch.rand(2, 9, 3)

In [23]:
sort_idx = torch.sort(a,descending=True)[1]

In [24]:
sort_idx

tensor([6, 3, 2, 1, 4, 7, 0, 5, 8])

In [25]:
_, sort_sort_idx = torch.sort(sort_idx)

In [26]:
sorted_a = a[torch.sort(-a)[1]]

In [27]:
sorted_a

tensor([8, 6, 5, 4, 3, 3, 2, 2, 2])

In [28]:
sort_sort_idx

tensor([6, 3, 2, 1, 4, 7, 0, 5, 8])

In [29]:
sorted_a[sort_sort_idx]

tensor([2, 4, 5, 6, 3, 2, 8, 3, 2])

In [30]:
a[torch.sort(-a)[1]]

tensor([8, 6, 5, 4, 3, 3, 2, 2, 2])

In [70]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size,n_layers, rnn_type = 'lstm', device = 'cuda'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = Embedding(input_size, embed_dim, PAD_IDX)
        self.rnn_type =  rnn_type
        self.dropout_in = nn.Dropout(p = 0.1)
        self.n_layers = n_layers
        self.device = device
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_size,batch_first=True,bidirectional=True, num_layers = self.n_layers, dropout = 0.2)
        elif rnn_type == 'lstm':
            self.rnn = LSTM(embed_dim, hidden_size, batch_first=True,bidirectional=True, num_layers = n_layers,dropout = 0.2)

    def forward(self, enc_inp, src_len):
        sorted_idx = torch.sort(src_len, descending=True)[1]
        orig_idx = torch.sort(sorted_idx)[1]
        embedded = self.embedding(enc_inp)
        bs = embedded.size(0)
        output = self.dropout_in(embedded)
#         print("encoded Input", output.shape)
#         output = torch.nn.utils.rnn.pack_padded_sequence(output, src_len, batch_first=True)
        if self.rnn_type == 'gru':
            hidden =  self.initHidden(bs)
            output, hidden = self.rnn(output, hidden)
#             output = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        elif self.rnn_type == 'lstm':
            hidden, c = self.initHidden(bs)
            sorted_output = output[sorted_idx]
            sorted_len = src_len[sorted_idx]
            packed_output = nn.utils.rnn.pack_padded_sequence(sorted_output, sorted_len.data.tolist(), batch_first = True)
            packed_outs, (hiddden, c) = self.rnn(packed_output,(hidden, c))
            hidden = hidden[:,orig_idx,:]
            c = c[:,orig_idx,:]
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_outs, padding_value=PAD_IDX, batch_first = True)
            output = output[orig_idx]
        hidden = hidden.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
        c = c.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
#         print(output.shape)
#         print(hidden.shape)
#         print(c.shape)
#             output = torch.nn.utils.rnn.pad_packed_sequence(output,batch_first=True)
#         print(hidden.shape)
#         print("rnn output", output.shape)
#         print("rnn hidden", hidden.shape)
#         hidden = hidden.transpose(0,1).contiguous().view(bs,-1)
#         hidden_1 = torch.tanh(self.linear1(hidden)).unsqueeze(0)
# #         print("rnn hidden_1", hidden_1.shape)
#         hidden_2 = torch.tanh(self.linear2(hidden)).unsqueeze(0)
# #         print("rnn hidden_2", hidden_2.shape)
#         hidden = torch.cat([hidden_1,hidden_2],dim = 0)
#         print("rnn hidden_final", hidden.shape)
#             print(hidden.shape)
#         last_layer = output[:,-1,:self.hidden_size]
#         first_layer = output[:,0,self.hidden_size:]
#         hidden = first_layer + last_layer
        return output, hidden, c

    def initHidden(self,bs):
        if self.rnn_type == 'gru' :
            return torch.zeros(self.n_layers*2, bs, self.hidden_size).to(self.device)
        elif self.rnn_type == 'lstm':
            return torch.zeros(self.n_layers*2,bs,self.hidden_size).to(self.device),torch.zeros(self.n_layers*2,bs,self.hidden_size).to(self.device)

class Attention_Module(nn.Module):
    def __init__(self, hidden_dim, output_dim, device = 'cuda'):
        super(Attention_Module, self).__init__()
        self.l1 = Linear(hidden_dim, output_dim, bias = False)
        self.l2 = Linear(hidden_dim+output_dim, output_dim, bias =  False)
        self.device = device
        
    def forward(self, hidden, encoder_outs, src_lens):
        ''' hiddden: bsz x hidden_dim
        encoder_outs: bsz x sq_len x encoder dim (output_dim)
        src_lens: bsz
        
        x: bsz x output_dim
        attn_score: bsz x sq_len'''
        x = self.l1(hidden)
        att_score = (encoder_outs.transpose(0,1) * x.unsqueeze(0)).sum(dim = 2)
        seq_mask = sequence_mask(src_lens, max_len = max(src_lens).item(), device = self.device).transpose(0,1)
        masked_att = seq_mask*att_score
        masked_att[masked_att==0] = -1e10
        attn_scores = F.softmax(masked_att, dim=0)
        x = (attn_scores.unsqueeze(2) * encoder_outs.transpose(0,1)).sum(dim=0)
        x = torch.tanh(self.l2(torch.cat((x, hidden), dim=1)))
        return x, attn_scores
        
class AttentionDecoderRNN(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, n_layers = 1, attention = True, device = 'cuda'):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        encoder_output_size = hidden_size
        self.embedding = Embedding(output_size, embed_dim, PAD_IDX)
        self.dropout = nn.Dropout(p=0.1)
        self.n_layers = n_layers
        self.device = device
        self.att_layer = Attention_Module(self.hidden_size, encoder_output_size,self.device) if attention else None
        self.layers = nn.ModuleList([
            LSTMCell(
                input_size=self.hidden_size + embed_dim if ((layer == 0) and attention) else embed_dim if layer == 0 else hidden_size,
                hidden_size=hidden_size,
            )
            for layer in range(self.n_layers)
        ])
#         if self.rnn_type == 'gru':
#             self.rnn = nn.GRU(self.hidden_size + embed_dim, hidden_size,batch_first=True, num_layers = self.n_layers, )
#         elif self.rnn_type == 'lstm':
#             self.rnn = nn.LSTM(self.hidden_size + embed_dim, hidden_size,batch_first=True, num_layers = self.n_layers)
        self.fc_out = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, dec_input,context_vector, prev_hiddens,prev_cs,encoder_outputs,src_len):
        bsz = dec_input.size(0)
        output = self.embedding(dec_input)
        output = self.dropout(output)
#         print("decoder Input embedded", output.shape)
        if self.att_layer is not None:
            cated_input = torch.cat([output.squeeze(1),context_vector], dim = 1)
        else:
            cated_input = output.squeeze(1)
#         print("cated_input",cated_input.shape)
        new_hiddens = []
        new_cs = []
        for i, rnn in enumerate(self.layers):
            hidden, c = rnn(cated_input, (prev_hiddens[i], prev_cs[i]))
            cated_input = self.dropout(hidden)
            new_hiddens.append(hidden.unsqueeze(0))
            new_cs.append(c.unsqueeze(0))
        new_hiddens = torch.cat(new_hiddens, dim = 0)
        new_cs = torch.cat(new_cs, dim = 0)

        # apply attention using the last layer's hidden state
        if self.att_layer is not None:
            out, attn_score = self.att_layer(hidden, encoder_outputs, src_len)
        else:
            out = hidden
            attn_score = None
        context_vec = out
        out = self.dropout(out)
        out_vocab = self.softmax(self.fc_out(out))

        return out_vocab, context_vec, new_hiddens, new_cs, attn_score
    
#     def initHidden(self,bs):
#         if self.rnn_type == 'gru' :
#             return None
#         elif self.rnn_type == 'lstm':
#             return torch.zeros(self.n_layers,bs,self.hidden_size).to(device)

In [32]:
def Embedding(num_embeddings, embedding_dim, padding_idx):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.uniform_(m.weight, -0.1, 0.1)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m


def LSTM(input_size, hidden_size, **kwargs):
    m = nn.LSTM(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def LSTMCell(input_size, hidden_size, **kwargs):
    m = nn.LSTMCell(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def Linear(in_features, out_features, bias=True, dropout=0):
    """Linear layer (input: N x T x C)"""
    m = nn.Linear(in_features, out_features, bias=bias)
    m.weight.data.uniform_(-0.1, 0.1)
    if bias:
        m.bias.data.uniform_(-0.1, 0.1)
    return m

In [78]:
def sequence_mask(sequence_length, max_len=None, device = 'cuda'):
    if max_len is None:
        max_len = sequence_length.max().item()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).repeat([batch_size,1])
    seq_range_expand = seq_range_expand.to(device)
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return (seq_range_expand < seq_length_expand).float()

In [34]:
# def encode_decode(encoder,decoder,data_en,data_de,src_len,tar_len,m_type,rand_num = 0.5):
#     use_teacher_forcing = True if random.random() < rand_num else False
# #     print("tar_len",tar_len)
#     bss = data_en.size(0)
#     en_hid = encoder.initHidden(bss)
#     en_out,en_hid = encoder(data_en, en_hid)
#     max_tar_len_batch = max(tar_len).item()
#     decoder_hidden = en_hid
#     decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
# #     c = decoder.initHidden(bss)
# #     print("max_tar_len_batch",max_tar_len_batch)
#     if use_teacher_forcing:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)   
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             decoder_input = data_de[:,i].view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
#     else:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)
#             else:
#                 decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             topv, topi = decoder_output.topk(1)
# #             decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
#     return d_out, d_hid

In [35]:
# def encode_decode(encoder,decoder,data_en,data_de,src_len,tar_len,m_type,rand_num = 0.5):
#     use_teacher_forcing = True if random.random() < rand_num else False
# #     print("tar_len",tar_len)
#     bss = data_en.size(0)
#     en_out,en_hid,c = encoder(data_en, src_len)
#     max_tar_len_batch = max(tar_len).item()
#     decoder_hidden = en_hid
#     decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
# #     print("max_tar_len_batch",max_tar_len_batch)
#     if use_teacher_forcing:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden, c = decoder(decoder_input,decoder_hidden,en_out,src_len,c)   
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             decoder_input = data_de[:,i].view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
# #         print("softmax mat", d_out.shape)
#     else:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out,src_len,c)
#             else:
#                 decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             topv, topi = decoder_output.topk(1)
# #             decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
#     return d_out, d_hid

In [36]:
def encode_decode(encoder,decoder,data_en,data_de,src_len,tar_len,rand_num = 0.95, val = False):
    if not val:
        use_teacher_forcing = True if random.random() < rand_num else False
    #     print("tar_len",tar_len)
        bss = data_en.size(0)
        en_out,en_hid,en_c = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out.size(-1))).to(device)
        if use_teacher_forcing:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
                d_out.append(out_vocab.unsqueeze(-1))
                decoder_input = data_de[:,i].view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
    #         print("softmax mat", d_out.shape)
        else:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
                d_out.append(out_vocab.unsqueeze(-1))
                topv, topi = out_vocab.topk(1)
                decoder_input = topi.squeeze().detach().view(-1,1)
    #             decoder_input = topi.squeeze().view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
        return d_out
    else:
        encoder.eval()
        decoder.eval()
        bss = data_en.size(0)
        en_out,en_hid,en_c = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out.size(-1))).to(device)
        d_out = []
        for i in range(max_tar_len_batch):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens,prev_cs, en_out,\
                                                                                    src_len)
            d_out.append(out_vocab.unsqueeze(-1))
            topv, topi = out_vocab.topk(1)
            decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
        d_out = torch.cat(d_out,dim=-1)
        return d_out

In [37]:
def flatten_cel_loss(input,target,nll):
    input = input.transpose(1,2)
    bs, sl = input.size()[:2]
    return nll(input.contiguous().view(bs*sl,-1),target.contiguous().view(-1))

In [38]:
def train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, loss_fun,m_type, dataloader, en_lang,\
                num_epochs=60, val_every = 1, train_bleu_every = 10,clip = 0.1, rm = 0.8, enc_scheduler = None,\
               dec_scheduler = None):
    best_score = 0
    best_bleu = 0
    loss_hist = {'train': [], 'val_train': []}
    bleu_hist = {'train': [], 'validate': []}
    best_encoder_wts = None
    best_decoder_wts = None
#     train_sorted = train.sort_values('en_len', ascending = False)
    phases = ['train','val_train']
#     phases = ['train']
    for epoch in range(num_epochs):
#         bs_dict = {'train':128,'validate':1, 'train_val':1,'val_train':128}
#         train_used = shuffle_sorted_batches(train_sorted, bs_dict['train'])
# #         train_used = train.iloc[:50]
# #         train_used = train
#         collate_fn_dict = {'train':vocab_collate_func, 'validate':vocab_collate_func_val,\
#                            'train_val':vocab_collate_func_val,'val_train':vocab_collate_func}
#         transformed_dataset = {'train': Vietnamese(train_used),
#                                'validate': Vietnamese(val, val = True),
#                                'train_val':Vietnamese(train.iloc[:50], val = True),
#                                'val_train':Vietnamese(val)
#                                                        }

#         dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=collate_fn_dict[x],
#                             shuffle=True, num_workers=0) for x in ['train', 'validate', 'train_val','val_train']}
        for ex, phase in enumerate(phases):
            start = time.time()
            total = 0
            top1_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                encoder.train()
                decoder.train()
            else:
                encoder.eval()
                decoder.eval()
            for data in dataloader[phase]:
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                encoder_i = data[0].to(device)
                decoder_i = data[1].to(device)
                src_len = data[2].to(device)
                tar_len = data[3].to(device)
#                 print(encoder_i.shape)
                if phase == 'val_train':                
                    out = encode_decode(encoder,decoder,encoder_i,decoder_i,src_len,tar_len,rand_num=rm,val = True )
                else:
                    out = encode_decode(encoder,decoder,encoder_i,decoder_i,src_len,tar_len,rand_num=rm,val = False )
                N = decoder_i.size(0)
#                 out = out.view(-1,en_lang.n_words)
#                 decoder_i = decoder_i.view(-1)
#                 if loss_fun == 'masked_cel':
#                     loss = masked_cross_entropy(out.float(), decoder_i.long(), tar_len)
#                 else:
#                     loss = flatten_cel_loss(out.float(), decoder_i.long(),loss_fun)
                loss = loss_fun(out.float(), decoder_i.long())
                running_loss += loss.item() * N
                
                total += N
                if phase == 'train':
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
                    torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
                    encoder_optimizer.step()
                    decoder_optimizer.step()
                    
            epoch_loss = running_loss / total 
            loss_hist[phase].append(epoch_loss)
            print("epoch {} {} loss = {}, time = {}".format(epoch, phase, epoch_loss,
                                                                           time.time() - start))
#             if epoch%train_bleu_every ==0:
#                 train_loss, train_bleu_score = validation(encoder,decoder, dataloader['train'],loss_fun, en_lang,max_len,m_type)
#                 bleu_hist['train'].append(train_bleu_score)
#                 print("Train BLEU = ", train_bleu_score)
        if (enc_scheduler is not None) and (dec_scheduler is not None):
            enc_scheduler.step(epoch_loss)
            dec_scheduler.step(epoch_loss)
        if epoch%val_every == 0:
            val_bleu_score = validation_new(encoder,decoder, dataloader['validate'], en_lang, m_type)
            bleu_hist['validate'].append(val_bleu_score)
            print("validation BLEU = ", val_bleu_score)
            if val_bleu_score > best_bleu:
                best_bleu = val_bleu_score
                best_encoder_wts = encoder.state_dict()
                best_decoder_wts = decoder.state_dict()
        print('='*50)
    encoder.load_state_dict(best_encoder_wts)
    decoder.load_state_dict(best_decoder_wts)
    print("Training completed. Best BLEU is {}".format(best_bleu))
    return encoder,decoder,loss_hist,bleu_hist

In [39]:
# def validation_new(encoder, decoder, val_dataloader, lang_en,m_type):
#     encoder.train(False)
#     decoder.train(False)
#     pred_corpus = []
#     true_corpus = []
#     running_loss = 0
#     running_total = 0
#     bl = BLEU_SCORE()
#     for data in val_dataloader:
#         encoder_i = data[0].to(device)
#         src_len = data[2].to(device)
#         bs,sl = encoder_i.size()[:2]
#         en_hid = encoder.initHidden(bs)
#         en_out,en_hid = encoder(encoder_i, en_hid)
#         decoder_hidden = en_hid
#         decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
# #         c = decoder.initHidden(bs)
#         d_out = []
#         for i in range(sl+20):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
#             d_out.append(topi.item())
#             if topi.item() == EOS_token:
#                 break
#         d_hid = decoder_hidden
        
#         true_corpus.append(data[-1])
#         pred_sent = convert_id_list_2_sent(d_out,lang_en)
#         pred_corpus.append(pred_sent)
# #         print("True Sentence:",data[-1])
# #         print("Pred Sentence:", pred_sent)
# #         print('-*'*50)
#     score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
#     return score

In [40]:
# def validation_new(encoder, decoder, val_dataloader, lang_en,m_type):
#     encoder.train(False)
#     decoder.train(False)
#     pred_corpus = []
#     true_corpus = []
#     running_loss = 0
#     running_total = 0
#     bl = BLEU_SCORE()
#     for data in val_dataloader:
#         encoder_i = data[0].to(device)
#         src_len = data[2].to(device)
#         bs,sl = encoder_i.size()[:2]
#         en_out,en_hid = encoder(encoder_i,src_len)
#         decoder_hidden = en_hid
#         decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
#         c = decoder.initHidden(bs)
#         d_out = []
#         for i in range(sl+20):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out,src_len,c)
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
#             d_out.append(topi.item())
#             if topi.item() == EOS_token:
#                 break
#         d_hid = decoder_hidden
        
#         true_corpus.append(data[-1])
#         pred_sent = convert_id_list_2_sent(d_out,lang_en)
#         pred_corpus.append(pred_sent)
# #         print("True Sentence:",data[-1])
# #         print("Pred Sentence:", pred_sent)
# #         print('-*'*50)
#     score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
#     return score

In [309]:
def validation_new(encoder, decoder, val_dataloader, lang_en,m_type, verbose = False, replace_unk = False):
    encoder.eval()
    decoder.eval()
    pred_corpus = []
    true_corpus = []
    running_loss = 0
    running_total = 0
    bl = BLEU_SCORE()
    for data in val_dataloader:
        encoder_i = data[0].to(device)
        src_len = data[2].to(device)
        bs,sl = encoder_i.size()[:2]
        en_out,en_hid,en_c = encoder(encoder_i,src_len)
        max_src_len_batch = max(src_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
        prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
        d_out = []
        for i in range(sl*2):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens,prev_cs, en_out,\
                                                                                    src_len)
            topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
            d_out.append(topi.item())
            decoder_input = topi.squeeze().detach().view(-1,1)
            if topi.item() == EOS_token:
                break
        

#         true_sent = convert_id_list_2_sent(data[1][0],lang_en)
#         true_corpus.append(true_sent)
        if replace_unk:
            true_sent = convert_id_list_2_sent(data[1][0],lang_en)
            true_corpus.append(true_sent)
        else:
            true_corpus.append(data[-1])
        pred_sent = convert_id_list_2_sent(d_out,lang_en)
        pred_corpus.append(pred_sent)
        if verbose:
            print("True Sentence:",data[-1])
            print("Pred Sentence:", pred_sent)
            print('-*'*50)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
    return score

In [308]:
def validation_beam_search(encoder, decoder, val_dataloader,lang_en,beam_size, verbose = False,\
                           device = 'cuda', replace_unk = False):
    encoder.eval()
    decoder.eval()
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    pred_corpus = []
    true_corpus = []
    running_loss = 0
    running_total = 0
    bl = BLEU_SCORE()
    j = 0
    for data in val_dataloader:
#         print(j)
        encoder_i = data[0].to(device)
        src_len = data[2].to(device)
        bs,sl = encoder_i.size()[:2]
        en_out,en_hid,en_c = encoder(encoder_i,src_len)
        max_src_len_batch = max(src_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
        prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
        list_decoder_input = [None]*beam_size
        beam_stop_flags = [False]*beam_size
        beam_score = torch.zeros((bs,beam_size)).to(device)
        list_d_outs = [[] for _ in range(beam_size)]
        select_beam_size = beam_size
        for i in range(sl+20):
            if i == 0:
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens,prev_cs, en_out,\
                                                                                    src_len)
                bss, vocab_size = out_vocab.size()
                topv, topi = out_vocab.topk(beam_size)
                list_prev_output = [prev_output]*beam_size
                list_prev_hiddens = [prev_hiddens]*beam_size
                list_prev_cs = [prev_cs]*beam_size
                for b in range(beam_size):
                    beam_score[0][b] = topv[0][b].item()
                    list_decoder_input[b] = topi[0][b].squeeze().detach().view(-1,1)
                    list_d_outs[b].append(topi[0][b].item())
                    if topi[0][b].item() == EOS_token:
                        beam_stop_flags[b] = True
            else:
                beam_out_vocab = [None]*beam_size
                temp_out = [None]*beam_size
                temp_hid = [None]*beam_size
                temp_c = [None]*beam_size
                prev_d_outs = copy.deepcopy(list_d_outs)
                for b in range(beam_size):
                    if not beam_stop_flags[b]:
                        beam_out_vocab[b], temp_out[b], temp_hid[b], temp_c[b], attention_score =\
                            decoder(list_decoder_input[b],list_prev_output[b],list_prev_hiddens[b],list_prev_cs[b],\
                                    en_out,src_len)
                        beam_out_vocab[b] = beam_out_vocab[b] + beam_score[0][b]
                    if beam_stop_flags[b]:
                        beam_out_vocab[b] = torch.zeros(bss,vocab_size).fill_(float('-inf')).to(device)
                beam_out_vocab = torch.cat(beam_out_vocab,dim = 1)
                
                topv, topi = beam_out_vocab.topk(beam_size)
                id_for_hid = topi//vocab_size
                topi_idx = topi%vocab_size
                for b in range(beam_size):
                    if not beam_stop_flags[b]:
                        beam_score[0][b] = topv[0][b].item()
                        list_decoder_input[b] = topi_idx[0][b].squeeze().detach().view(-1,1)
                        list_d_outs[b] = copy.deepcopy(prev_d_outs[id_for_hid[0][b]])
                        list_d_outs[b].append(topi_idx[0][b].item())
                        if topi_idx[0][b].item() == EOS_token:
                            beam_stop_flags[b] = True
                        else:
                            list_prev_output[b] = temp_out[id_for_hid[0][b]]
                            list_prev_hiddens[b] = temp_hid[id_for_hid[0][b]]
                            list_prev_cs[b] = temp_c[id_for_hid[0][b]]
                if all(beam_stop_flags):
                    break
        id_max_score = np.argmax(beam_score)
        d_out = list_d_outs[id_max_score]
        if replace_unk:
            true_sent = convert_id_list_2_sent(data[1][0],lang_en)
            true_corpus.append(true_sent)
        else:
            true_corpus.append(data[-1])
        pred_sent = convert_id_list_2_sent(d_out,lang_en)
        pred_corpus.append(pred_sent)
        if verbose:
            print("True Sentence:",data[-1])
            print("Pred Sentence:", pred_sent)
            print('-*'*50)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
    return score

In [91]:
enc_cpu = EncoderRNN(vi_lang.n_words,512,512, 2, device = 'cuda').to(device)
dec_cpu = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2, attention = True, device = 'cuda').to(device)

In [297]:
enc_cpu.load_state_dict(torch.load('best_att_encoder_2.pth'))

In [298]:
dec_cpu.load_state_dict(torch.load('best_att_decoder_2.pth'))

In [75]:
# enc_cpu = enc_cpu.to('cpu')
# dec_cpu = dec_cpu.to('cpu')

In [299]:
validation_beam_search(enc_cpu,dec_cpu,dataloader['validate'],en_lang, 3, verbose=False, device = 'cuda')

26.133299858850144

In [305]:
start_time = time.time()
bleu_greedy = validation_new(enc_cpu,dec_cpu,dataloader['validate'],en_lang, 3, verbose=False)
end_time = time.time()
print('Time taken = ',end_time - start_time)

Time taken =  46.88684391975403


In [ ]:
start_time = time.time()
bleu_beam_3 = validation_beam_search(enc_cpu,dec_cpu,dataloader['validate'],en_lang, 5, verbose=False, device = 'cuda')
end_time = time.time()
print('Time taken = ',end_time - start_time)
print(bleu_beam_3)

In [302]:
enc_cpu.load_state_dict(torch.load('best_att_encoder_3.pth'))
dec_cpu.load_state_dict(torch.load('best_att_decoder_3.pth'))

In [303]:
validation_beam_search(enc_cpu,dec_cpu,dataloader['validate'],en_lang, 3, verbose=False, device = 'cuda')

27.433485312789212

In [54]:
def masked_cross_entropy(logits, target, length):
    logits = logits.transpose(1,2)
#     print("logits",logits.shape)
#     print("target",target.shape)
    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.contiguous().view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
#     log_probs_flat = F.log_softmax(logits_flat, dim = 1)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
#     print("tar_flat", target_flat.shape)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(logits_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask
    loss = losses.sum() / length.float().sum()
    return loss

In [55]:
criterion = 'masked_cel'

# Old Interesting performance

In [305]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

In [77]:
encoder = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
decoder = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2).to(device)

In [78]:
# encoder = OldEncoderRNN(vi_lang.n_words,512,bi = True).to(device)
# decoder = OldAttentionDecoderRNN(512,en_lang.n_words, True, MAX_LEN, attention_type='t2').to(device)

In [79]:
encoder.rnn

LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)

In [80]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=1e-4)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=1e-4)

In [81]:
enc, dec, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, encoder, decoder, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 50)

epoch 0 train loss = 5.654132865225413, time = 341.1948525905609
validation BLEU =  0.0
epoch 1 train loss = 5.45065745284289, time = 341.9193711280823
validation BLEU =  0.0
epoch 2 train loss = 5.210996340336155, time = 342.40592312812805
validation BLEU =  0.0
epoch 4 train loss = 4.809566239472573, time = 341.6780881881714
validation BLEU =  0.0
epoch 5 train loss = 4.63142532622266, time = 341.56193590164185
validation BLEU =  0.0
epoch 6 train loss = 4.520703522670287, time = 341.8052592277527
validation BLEU =  0.007257456178771249
epoch 7 train loss = 4.4197971115632395, time = 341.1301064491272
validation BLEU =  0.00778988176186667
epoch 8 train loss = 4.318348191080374, time = 341.3023076057434
validation BLEU =  0.00873574484162468
epoch 9 train loss = 4.237790871660339, time = 341.7362952232361
validation BLEU =  0.01489571311812371
epoch 10 train loss = 4.151050939980724, time = 341.4068603515625
validation BLEU =  0.019931957373692067
epoch 11 train loss = 4.091922725855

In [84]:
validation_new(enc,dec,dataloader['train_val'],en_lang, 'attention', verbose=True)

True Sentence: Rachel Pike : The science behind a climate headline
Pred Sentence: UKN UKN revolution .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .
Pred Sentence: In four minutes , the UKN UKN UKN UKN UKN of of of the the the the the the the , , , , the the the , , , , , , , , , , the the the the the the the the the the the
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .
Pred Sentence: I want you to know about the great challenges of the chal

True Sentence: And it takes weeks to perform our integrations .
Pred Sentence: It &apos;s takes a week of weeks .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: And we perform dozens of integrations in order to understand what &apos;s happening .
Pred Sentence: We need hundreds of these kinds of algorithms to do that .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: We also fly all over the world looking for this thing .
Pred Sentence: We &apos;re around around the world to to .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: I recently joined a field campaign in Malaysia . There are others .
Pred Sentence: I recently joined a UKN in the UKN . . . . . . .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: We found a glob

6.89490079655882

# Second with shuffled batches

In [61]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

In [50]:
encoder = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
decoder = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2, attention = True).to(device)

In [37]:
# encoder = OldEncoderRNN(vi_lang.n_words,512,bi = True).to(device)
# decoder = OldAttentionDecoderRNN(512,en_lang.n_words, True, MAX_LEN, attention_type='t2').to(device)

In [51]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr = 5e-3)
decoder_optimizer = optim.Adam(decoder.parameters(), lr = 5e-3)
# encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, factor = 0.5, min_lr=5e-5,  patience=0)
# decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, factor = 0.5, min_lr=5e-5,  patience=0)

In [52]:
enc, dec, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, encoder, decoder, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 15, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 6.663351734576734, time = 1114.1865501403809
epoch 0 val_train loss = 8.06595925051331, time = 6.069099426269531
validation BLEU =  1.3829197773400932
epoch 1 train loss = 6.219018471040969, time = 1113.4414727687836
epoch 1 val_train loss = 8.016192072197466, time = 6.1017372608184814
validation BLEU =  1.1039038541592752
epoch 2 train loss = 5.939835129506533, time = 1111.5073668956757
epoch 2 val_train loss = 7.930288785639622, time = 6.0234386920928955
validation BLEU =  1.9728202797441594
epoch 3 train loss = 5.336882101169395, time = 1106.1388936042786
epoch 3 val_train loss = 7.170681196808439, time = 6.007143020629883
validation BLEU =  2.5507477701856334
epoch 4 train loss = 4.53218613214774, time = 1105.0649061203003
epoch 4 val_train loss = 6.974173709797182, time = 6.019273042678833
validation BLEU =  3.7938947592150014
epoch 5 train loss = 4.203932680514588, time = 1105.1447207927704
epoch 5 val_train loss = 6.563302664350636, time = 6.0073864459991455

In [53]:
validation_new(enc,dec,dataloader['train_val'],en_lang, 'attention', verbose=True)

True Sentence: we run enormous models on supercomputers ; this is what i happen to do .
Pred Sentence: we published the code of the code ; it &apos;s the most beautiful thing .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and as you come around the banks in these valleys , the forces can get up to two gs .
Pred Sentence: when the light is flowing down , the river is able to move the UKN to the UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: because it &apos;s important to the atmospheric system , we go to all lengths to study this thing .
Pred Sentence: it &apos;s because it &apos;s important to the system , the system , we also have this system that we also have .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and one of the molecules i study is called isoprene , which is he

True Sentence: we blow it up and look at the pieces .
Pred Sentence: we showed it the same thing and the UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: that report was written by 620 scientists from 40 countries .
Pred Sentence: the study was written by the UKN from the national academy .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and on part of that field campaign we even brought an aircraft with us .
Pred Sentence: there are a thousand UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and when one student like me has some sort of inclination or understanding about that molecule , they write one scientific paper on the subject .
Pred Sentence: when i got the idea of the library , that &apos;s what that synthesis was , that they &apos;re going to be able to study that s

8.350793619403419

In [80]:
validation_new(enc,dec,dataloader['validate'],en_lang, 'attention', verbose=False)

21.62194078557701

In [54]:
encoder = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
decoder = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2, attention = True).to(device)

In [55]:
# encoder = OldEncoderRNN(vi_lang.n_words,512,bi = True).to(device)
# decoder = OldAttentionDecoderRNN(512,en_lang.n_words, True, MAX_LEN, attention_type='t2').to(device)

In [56]:
# encoder_optimizer = optim.Adam(encoder.parameters(), lr = 5e-4)
# decoder_optimizer = optim.Adam(decoder.parameters(), lr = 5e-4)
# encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
# enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-4,  patience=0)
# decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
# dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-4,  patience=0)

In [57]:
enc, dec, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, encoder, decoder, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 15, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 5.393191720143928, time = 1110.3711938858032
epoch 0 val_train loss = 6.350985495449993, time = 6.045475959777832
validation BLEU =  6.097490864225506
epoch 1 train loss = 3.7876384918800445, time = 1109.0049946308136
epoch 1 val_train loss = 6.497099796680246, time = 6.041832447052002
validation BLEU =  14.827929089259817
epoch 2 train loss = 3.2587029963508893, time = 1110.3191792964935
epoch 2 val_train loss = 6.220447167237098, time = 6.061037063598633
validation BLEU =  16.523216268913465
epoch 3 train loss = 3.1221366043516166, time = 1110.687434911728
epoch 3 val_train loss = 6.181964690752962, time = 6.036092281341553
validation BLEU =  17.562226160158012
epoch 4 train loss = 3.023102364547647, time = 1109.6038773059845
epoch 4 val_train loss = 6.166984209126855, time = 6.062610387802124
validation BLEU =  18.338058140978138
epoch 5 train loss = 3.008821606100434, time = 1111.0691175460815
epoch 5 val_train loss = 5.993662122672289, time = 6.037784814834595

In [58]:
validation_new(enc,dec,dataloader['train_val'],en_lang, 'attention', verbose=True)

True Sentence: so maybe you took a similar aircraft to get here today .
Pred Sentence: you can have on a future where you come here .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and the scientists have to be completely harnessed in in order to make measurements while they &apos;re on board .
Pred Sentence: scientists have to be UKN completely on the chair to make the UKN on the airplane .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: we found a global atmospheric watchtower there , in the middle of the rainforest , and hung hundreds of thousands of dollars worth of scientific equipment off this tower , to look for isoprene , and of course , other things while we were there .
Pred Sentence: we found a global climate in which , right in the middle of the middle , and we &apos;ve UKN hundreds of thousands of dollars from the UKN to find UKN , and of

True Sentence: we run enormous models on supercomputers ; this is what i happen to do .
Pred Sentence: we run huge models of these computers ; this is my work .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and because it &apos;s so much stuff , it &apos;s really important for the atmospheric system .
Pred Sentence: because the amount of carbon is large , it means important to the atmosphere system .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: i &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .
Pred Sentence: i want to show you about the great UKN of science that have been used to make the UKN you see on the page .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: we also fly all over the world look

12.301247761128733

In [63]:
torch.save(enc.state_dict(), 'best_att_encoder.pth')
torch.save(dec.state_dict(), 'best_att_decoder.pth')

In [84]:
encoder_optimizer = optim.Adam(enc.parameters(), lr = 5e-4)
decoder_optimizer = optim.Adam(dec.parameters(), lr = 5e-4)
# encoder_optimizer = optim.SGD(encoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)
# decoder_optimizer = optim.SGD(decoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)

In [85]:
enc_2, dec_2, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, enc, dec, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 5, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 2.881824669845856, time = 1110.7850728034973
epoch 0 val_train loss = 6.033290113936463, time = 6.028157472610474
validation BLEU =  22.55434033493915
epoch 1 train loss = 2.4991769172360256, time = 1111.6410603523254
epoch 1 val_train loss = 5.55787919522836, time = 6.039398670196533
validation BLEU =  22.645028117048412
epoch 2 train loss = 2.3047653842573417, time = 1111.2298455238342
epoch 2 val_train loss = 5.991711738357784, time = 6.075587749481201
validation BLEU =  25.40564173998178
epoch 3 train loss = 2.0650567520692804, time = 1111.0621933937073
epoch 3 val_train loss = 6.063444457971711, time = 6.115215063095093
validation BLEU =  26.568294217375808
epoch 4 train loss = 1.8991269282373244, time = 1111.121065378189
epoch 4 val_train loss = 5.998856958154624, time = 6.0773890018463135
validation BLEU =  26.493159328131288
Training completed. Best BLEU is 26.568294217375808


In [86]:
torch.save(enc_2.state_dict(), 'best_att_encoder_2.pth')
torch.save(dec_2.state_dict(), 'best_att_decoder_2.pth')

In [89]:
validation_new(enc_2,dec_2,dataloader['validate'],en_lang, 'attention', verbose=False)

25.067320730375243

In [98]:
enc_2 = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
dec_2 = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2, attention = True).to(device)

In [99]:
enc_2.load_state_dict(torch.load('best_att_encoder_2.pth'))

In [100]:
dec_2.load_state_dict(torch.load('best_att_decoder_2.pth'))

In [33]:
encoder_optimizer = optim.Adam(enc_2.parameters(), lr = 7e-5)
decoder_optimizer = optim.Adam(dec_2.parameters(), lr = 7e-5)
# encoder_optimizer = optim.SGD(encoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)
# decoder_optimizer = optim.SGD(decoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)

In [34]:
enc_2, dec_2, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, enc_2, dec_2, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 5, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 1.8260680445180335, time = 1037.8615787029266
epoch 0 val_train loss = 6.089784148363661, time = 5.407512187957764
validation BLEU =  25.366019799426176
epoch 1 train loss = 1.7608592886574477, time = 1038.3074650764465
epoch 1 val_train loss = 5.998265737238743, time = 5.37864351272583
validation BLEU =  25.280588277516443
epoch 2 train loss = 1.6716099943854263, time = 1037.1525812149048
epoch 2 val_train loss = 6.164014130360696, time = 5.369064569473267
validation BLEU =  25.1663732632782
epoch 3 train loss = 1.6779262772508994, time = 1037.7245879173279
epoch 3 val_train loss = 6.211051822084734, time = 5.37527322769165
validation BLEU =  25.370267338902842
epoch 4 train loss = 1.6805312998572008, time = 1038.8336567878723
epoch 4 val_train loss = 6.1532688231122235, time = 5.375488758087158
validation BLEU =  24.997773401216595
Training completed. Best BLEU is 25.370267338902842


In [56]:
enc_2 = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
dec_2 = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2, attention = True).to(device)

In [57]:
enc_2.load_state_dict(torch.load('best_att_encoder_2.pth'))

In [58]:
dec_2.load_state_dict(torch.load('best_att_decoder_2.pth'))

In [59]:
encoder_optimizer = optim.Adam(enc_2.parameters(), lr = 7e-5)
decoder_optimizer = optim.Adam(dec_2.parameters(), lr = 7e-5)
# encoder_optimizer = optim.SGD(encoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)
# decoder_optimizer = optim.SGD(decoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)

In [62]:
enc_2, dec_2, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, enc_2, dec_2, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 5, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 1.7527952763680839, time = 1034.6966149806976
epoch 0 val_train loss = 6.016703680862764, time = 5.353416681289673
validation BLEU =  25.480735516107423
epoch 1 train loss = 1.7590020810745766, time = 1034.8196697235107
epoch 1 val_train loss = 6.045576048950291, time = 5.352719783782959
validation BLEU =  25.11227801933653
epoch 2 train loss = 1.6927039314589836, time = 1033.9458301067352
epoch 2 val_train loss = 6.146998235479891, time = 5.350436210632324
validation BLEU =  25.38545849576757
epoch 3 train loss = 1.7002392874466004, time = 1034.6097462177277
epoch 3 val_train loss = 6.043364785071051, time = 5.352215051651001
validation BLEU =  25.009778906188913
epoch 4 train loss = 1.641774711446003, time = 1034.276869058609
epoch 4 val_train loss = 6.162587725401677, time = 5.355162620544434
validation BLEU =  25.122648931435688
Training completed. Best BLEU is 25.480735516107423


In [63]:
torch.save(enc_2.state_dict(), 'best_att_encoder_3.pth')
torch.save(dec_2.state_dict(), 'best_att_decoder_3.pth')

In [146]:
test_example = 'họ sống hạnh phúc mãi mãi về sau .'.split(' ')

In [147]:
test_idx = []
for i in test_example:
    try:
        test_idx.append(vi_lang.word2index[i])
    except:
        test_idx.append(UNK_IDX)

In [148]:
test_idx

[79, 503, 10076, 9725, 1009, 1009, 4, 141, 6]

In [149]:
a = torch.from_numpy(np.array(test_idx)).unsqueeze(0).to(device)

In [150]:

encoder_i = a.to(device)
src_len = torch.tensor([a.size(1)]).to(device)
bs,sl = encoder_i.size()[:2]
en_out,en_hid,en_c = enc_2(encoder_i,src_len)
max_src_len_batch = max(src_len).item()
prev_hiddens = en_hid
prev_cs = en_c
decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
d_out = []
for i in range(sl*2):
    out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = dec_2(decoder_input,prev_output, \
                                                                            prev_hiddens,prev_cs, en_out,\
                                                                            src_len)
    topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
    d_out.append(topi.item())
    decoder_input = topi.squeeze().detach().view(-1,1)
    if topi.item() == EOS_token:
        break


print(d_out)
pred_sent = convert_id_list_2_sent(d_out,en_lang)

print("Pred Sentence:", pred_sent)
print('-*'*50)

[35, 426, 1051, 762, 8, 1]
Pred Sentence: they live happiness forever .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*


In [106]:
validation_new(enc_2,dec_2,dataloader['validate'],en_lang, 'attention', verbose=False)

25.067320730375243

In [35]:
validation_new(enc_2,dec_2,dataloader['validate'],en_lang, 'attention', verbose=True)

True Sentence: i want to change that perception . i want to change those feelings you have for these birds , because they need our sympathy . they really do .
Pred Sentence: i want to change that consciousness . i want to change the feelings that you give these birds , because they need to be in our compassion . they really need to be really .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: it was on a strip of land that we call a parkway .
Pred Sentence: it &apos;s on a block of land that we call the UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and i was scared .
Pred Sentence: i was very scared .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: he sat next to me on the new york city subway , and he started chatting me up .
Pred Sentence: he sat next to me in the new york city

True Sentence: that &apos;s double the amount of people taken from africa during the entire trans-atlantic slave trade .
Pred Sentence: twice the number of people were taken out of africa during the UKN trade of the atlantic .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: however , there are 6.3 million people worldwide who have the disease , and they have to live with incurable weakness , tremor , rigidity and the other symptoms that go along with the disease , so what we need are objective tools to detect the disease before it &apos;s too late .
Pred Sentence: however , UKN million people have a disease , and they have to live in debilitating , UKN , UKN UKN and other symptoms that go with this disease . what we need is tools that are available to detect the disease before we &apos;re too late .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: thank

True Sentence: but this time , i would see the skeletons hidden in the closet .
Pred Sentence: but this time , i want to see the truth that &apos;s being UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: this is manuru . when his father died , his uncle trafficked him to work with him in the mines .
Pred Sentence: this is UKN . when his father died , he was brought to the mine with the same door .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: they are very graceful . charles darwin said he changed his mind because he watched them fly effortlessly without energy in the skies .
Pred Sentence: they &apos;re very graceful . charles darwin said he had seen his opinion because he saw that the fly was not needed to get a little bit of UKN on the sky .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Tr

True Sentence: many of them drown .
Pred Sentence: many of them are drowning .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: because it turns out that i &apos;m actually a very typical domestic violence victim and a typical domestic violence survivor .
Pred Sentence: because the truth that i &apos;m a senior victim of family violence is ending to me as a typical survivor of domestic violence .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: almost two years have passed since the libyan revolution broke out , inspired by the waves of mass mobilization in both the tunisian and the egyptian revolutions .
Pred Sentence: almost two years have been passed since the UKN in libya , which was UKN by the UKN wave of UKN waves from the from the from the in tunisia and the egyptian .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-

True Sentence: i met these boys at five in the morning , when they were hauling in the last of their nets , but they had been working since 1 a.m.
Pred Sentence: i met these boys at five o &apos;clock in the morning , when they &apos;re pulling the last round of the grid , but they &apos;ve done a lot of a light hour .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: because this time it wasn &apos;t coming up .
Pred Sentence: because this time is not .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: for the last 28 years , i &apos;ve been documenting indigenous cultures in more than 70 countries on six continents , and in 2009 i had the great honor of being the sole exhibitor at the vancouver peace summit .
Pred Sentence: in the 28 years , i &apos;ve collected a local culture of local culture in more than 70 continents , and in 2009 i had the best hono

...

Pred Sentence: the shadows were off and UKN lions made them think i was walking around the UKN , but actually i was sleeping on my bed .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: so why did i do that ?
Pred Sentence: why do i do that ?
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: i lost all hope .
Pred Sentence: i was completely UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and my father -- that &apos;s him -- he was the first ever in his family to receive an education .
Pred Sentence: and this is my father who was the first person in his family &apos;s family .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: so the city planners , they get together and they figure they &apos;re going to change the nam

True Sentence: after they finished questioning me , one official said to another , &quot; this was a false report .
Pred Sentence: after the end of it , one of the cops said to the other , this was a wrong UKN .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: there she is .
Pred Sentence: here she is .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: and some even said , this would be stasi 2.0 .
Pred Sentence: and some people even say , it &apos;s like UKN 2.0 .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: if you had told me that this smart , funny , sensitive man who adored me would one day dictate whether or not i wore makeup , how short my skirts were , where i lived , what jobs i took , who my friends were and where i spent christmas , i would have laughed at you , because there

True Sentence: a hundred and fifty years ago , an agricultural slave cost about three times the annual salary of an american worker .
Pred Sentence: 150 years ago , a UKN cost three years of income per year .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: thus we are told that &quot; my mercy encompasses all things . &quot;
Pred Sentence: so we were taught that &quot; tolerance of the world . &quot;
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: the funny thing about sustainability , you have to sustain it .
Pred Sentence: the interesting thing about sustainability , is that you have to maintain it .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: over 85 percent of abusers are men , and domestic abuse happens only in intimate , interdependent , long-term relationships , in other wor

True Sentence: &quot; before i die , i want to be tried for piracy . &quot;
Pred Sentence: &quot; before i die , i want to try UKN . &quot;
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: i &apos;m just one woman with a story to tell .
Pred Sentence: i &apos;m just a woman who wants to share her story .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: deadened by monotony and exhaustion , they work silently , doing this task over and over for 16 or 17 hours a day .
Pred Sentence: UKN by UKN and UKN , they silence , and they go over and over and over and over and 17 , every day .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: he kept one in the glove compartment of our car .
Pred Sentence: he holds a tree in the UKN of our car .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

KeyboardInterrupt: 

In [67]:
pickle.dump(en_lang,open('en_lang_class','wb'))
pickle.dump(vi_lang,open('vi_lang_class','wb'))

# Without attention

In [67]:
encoder_wo_att = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
decoder_wo_att = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2, attention = False).to(device)

In [68]:
# encoder = OldEncoderRNN(vi_lang.n_words,512,bi = True).to(device)
# decoder = OldAttentionDecoderRNN(512,en_lang.n_words, True, MAX_LEN, attention_type='t2').to(device)

In [81]:
encoder_optimizer = optim.Adam(encoder_wo_att.parameters(), lr = 5e-4)
decoder_optimizer = optim.Adam(decoder_wo_att.parameters(), lr = 5e-4)
# encoder_optimizer = optim.SGD(encoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)
# decoder_optimizer = optim.SGD(decoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-5,factor = 0.5,  patience=0)

In [82]:
encoder_wo_att, decoder_wo_att, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, encoder_wo_att, decoder_wo_att, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 15, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 3.1821780225911347, time = 636.8962428569794
epoch 0 val_train loss = 5.376200480415994, time = 3.1069211959838867
validation BLEU =  8.985798203763085
epoch 1 train loss = 2.9451840752333562, time = 636.574697971344
epoch 1 val_train loss = 5.234533785269464, time = 3.106020212173462
validation BLEU =  8.850306649681523
epoch 2 train loss = 2.7331653086472447, time = 636.5499529838562
epoch 2 val_train loss = 5.391618499996534, time = 3.1056020259857178
validation BLEU =  9.46471026614434
epoch 3 train loss = 2.465097379227344, time = 636.6200149059296
epoch 3 val_train loss = 5.413536125929197, time = 3.1518595218658447
validation BLEU =  9.875785403841025
epoch 4 train loss = 2.3155312629657274, time = 636.8171179294586
epoch 4 val_train loss = 5.7467264271685, time = 3.0996038913726807
validation BLEU =  10.95616690740053
epoch 5 train loss = 2.2263531891877975, time = 636.8281514644623
epoch 5 val_train loss = 5.67048430518021, time = 3.1139612197875977
valida

In [83]:
validation_new(encoder_wo_att, decoder_wo_att,dataloader['train_val'],en_lang, 'attention', verbose=True)

True Sentence: and when one student like me has some sort of inclination or understanding about that molecule , they write one scientific paper on the subject .
Pred Sentence: when i was asked when i was interested in language , it was kind of a little bit of a , , , to to of
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: it &apos;s a huge amount of stuff . it &apos;s equal to the weight of methane .
Pred Sentence: it &apos;s a huge amount of carbon dioxide , a bit of the UKN of of
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: we also fly all over the world looking for this thing .
Pred Sentence: we also traveled the world to look at this molecule .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: i recently joined a field campaign in malaysia . there are others .
Pred Sentence: i re

True Sentence: and in each one of these research areas , of which there are even more , there are phd students , like me , and we study incredibly narrow topics , things as narrow as a few processes or a few molecules .
Pred Sentence: and each of these interactions is done in more than 100 , and they have been my graduate students , and they &apos;re really interested in some interesting , very , very , , and
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: recently the headlines looked like this when the intergovernmental panel on climate change , or ipcc , put out their report on the state of understanding of the atmospheric system .
Pred Sentence: the headlines have seen this as if the UKN of the the the the the , , , the the the the the the . . . .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: in 4 minutes , atmospheric chemist rachel pike provide

10.827820048043678